# Подсчет частотности пар товаров в продуктовых чеках

В файле содержится информация о покупках людей.

* id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* Товар – наименование товара
* Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

In [ ]:
%%capture
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `db` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE db;'

In [ ]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import os
from sqlalchemy import create_engine
import numpy as np
import random

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
user = 'postgres'
password = 'postgres'
host = 'localhost'
post = 5432
db = 'db'

In [ ]:
point = 'postgresql://{}:{}@{}:{}/{}'.format(user, password, host, post, db)
con =  create_engine(point)

In [ ]:
def create_table() -> None:
    """Создать таблицу в БД """
    try:
        conn = psycopg2.connect(point)
        cursor = conn.cursor()
        cursor.execute(
            """CREATE TABLE IF NOT EXISTS sales (check_number SERIAL NOT NULL,
				                                         product_name VARCHAR (25) NOT NULL,
				                                         quantity SERIAL NOT NULL);"""
                       )
        conn.commit()
    except (Exception, Error) as error:
        print("Ошибка при работе с PostgreSQL", error)
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с PostgreSQL закрыто")

In [ ]:
create_table()

Соединение с PostgreSQL закрыто


In [ ]:
list_check_number = []
list_product_name = []
list_quantity = []
for _ in range(10000):
  list_check_number.append(random.randint(1,1000))

  current_product = "pr_" + str(random.randint(1,100))
  list_product_name.append(current_product)

  list_quantity.append(random.randint(1,10))

In [ ]:
d = {'check_number': list_check_number, 'product_name': list_product_name, 'quantity': list_quantity}

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
df.head()

,check_number,product_name,quantity
0,461,pr_21,2
1,514,pr_36,4
2,29,pr_47,3
3,123,pr_47,3
4,22,pr_63,6


In [ ]:
df.to_sql('sales', con, index=False, if_exists='replace', method='multi')

In [ ]:
def select_postgresql(sql: str):
    """Запрос данных из БД"""
    conn = psycopg2.connect(point)
    return pd.read_sql(sql, conn)

In [ ]:
sql = """select s.* from sales as s where s.check_number = 1 order by s.product_name"""

In [ ]:
select_postgresql(sql)

,check_number,product_name,quantity
0,1,pr_100,8
1,1,pr_11,2
2,1,pr_20,4
3,1,pr_55,5
4,1,pr_63,1
5,1,pr_66,3
6,1,pr_88,8
7,1,pr_89,1
8,1,pr_97,6
9,1,pr_99,4


In [ ]:
sql = """with tbl as (select s.*, row_number() over (order by s.product_name) as id
                      from (select distinct s.product_name
                          from sales as s) as s),
              tbl_two_products as (select t1.product_name as product_1, t2.product_name as product_2
                                   from tbl as t1, tbl as t2
                                   where (t1.id<>t2.id) and (t1.id<t2.id)),              
             tbl_check_group_product_name as (select s.check_number, array_agg(s.product_name) as list_product_name
                                              from sales as s
                                              group by s.check_number)
              select gr.product_1, gr.product_2, count(gr.list_product_name) as amount
              from (select pr.product_1, pr.product_2, ch.list_product_name
                    from tbl_two_products as pr left join tbl_check_group_product_name as ch 
                                                on cast(ch.list_product_name as text) like ('%'|| cast(pr.product_1 as text) || '%')
                                                    and cast(ch.list_product_name as text) like ('%'|| cast(pr.product_2 as text) || '%')
               where ch.list_product_name is not null) as gr
               group by gr.product_1, gr.product_2
               order by count(gr.list_product_name) desc
               limit 5"""

In [ ]:
select_postgresql(sql)

,product_1,product_2,amount
0,pr_1,pr_9,466
1,pr_1,pr_6,465
2,pr_3,pr_4,460
3,pr_1,pr_8,460
4,pr_4,pr_8,460
